In [1]:
import pandas as pd

In [57]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [64]:
glu_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Glu_FA_17.xlsx'
df_glu1 = load_data(glu_path)
glu_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/18_Glu_A.xlsx'
df_glu1_alkb = load_data(glu_path)

In [75]:
df_glu1 = df_glu1[df_glu1.Mass>1500]
df_glu1_alkb = df_glu1_alkb[df_glu1_alkb.Mass>1500]

In [70]:
glu_path = '/Users/xyuan/Documents/ChenLab/8May2021/UC_glu_5_rosa.xlsx'
df_glu5 = load_data(glu_path)
glu_path = '/Users/xyuan/Documents/ChenLab/8May2021/UC_glu_6_rosa.xlsx'
df_glu5_alkb = load_data(glu_path)
df_glu5.shape, df_glu5_alkb.shape

((916, 5), (1445, 5))

In [82]:
df_glu5 = df_glu5[df_glu5.Mass>1500]
df_glu5_alkb = df_glu5_alkb[df_glu5_alkb.Mass>1500]

In [136]:
plotly_zone(df_glu5)

In [78]:
_, df_alkb1_only = diff_dfs(df_glu1, df_glu1_alkb)
df_alkb1_only.shape

(2157, 5)

In [84]:
_, df_alkb5_only = diff_dfs(df_glu5, df_glu5_alkb)
df_alkb5_only.shape

(648, 5)

In [101]:
_, df_alkb5_comm = comm_dfs(df_glu5, df_glu5_alkb)
df_alkb5_comm.shape

(218, 5)

In [115]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 16
params = [(df_glu1, df_alkb1_only, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [130]:
fig = px.scatter(df_shift_hits, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [144]:
df_sh1.sort_values('hit', ascending=False).reset_index(drop=True).iloc[:10].copy()

,shift,hit
0,-1,217
1,-15,204
2,1,202
3,-16,200
4,-14,199
5,5,180
6,-37,176
7,12,176
8,-2,174
9,33,174


In [88]:
# df_sh1 = df_shift_hits.copy()
df_sh5 = df_shift_hits.copy()

In [137]:
dfm_l, dfm_r = peer_dfs(df_glu5, df_alkb5_only, shift=1)
dfm_l.shape, dfm_r.shape

((83, 5), (82, 5))

In [141]:
shift = 1
# dfm_l = match_dfs(df_glu1, df_alkb1_only, shift=shift)
# dfm_r = match_dfs(df_alkb1_only, df_glu1, shift=-1*shift)
dfm_l, dfm_r = peer_dfs(df_glu1, df_alkb1_only, shift=shift)
dfm_l.shape, dfm_r.shape

((205, 5), (202, 5))

In [43]:
shift = 1
dfm_wild_l = match_dfs(df_glu1, df_glu1, shift=-1*shift)
dfm_wild_r = match_dfs(df_glu1, df_glu1, shift=shift)

dfm_alkb_l = match_dfs(df_glu1_alkb, df_glu1_alkb, shift=-1*shift)
dfm_alkb_r = match_dfs(df_glu1_alkb, df_glu1_alkb, shift=shift)
dfm_wild_l.shape, dfm_wild_r.shape, dfm_alkb_l.shape, dfm_alkb_r.shape

((398, 5), (391, 5), (892, 5), (879, 5))

In [105]:
df_glu5_extended = pd.concat([df_glu5, dfm_l]).drop_duplicates()
df_glu5_extended.shape

(438, 5)

In [142]:
bcr = base_calling_random(dfm_l, base_only=True)
plotly_basecalling(*bcr)
bcr = base_calling_random(dfm_r, base_only=True)
plotly_basecalling(*bcr)

mass_x (205,) mass_y (205,)


mass_x (202,) mass_y (202,)


In [111]:
plotly_basecalling(*bcr_ext)

In [145]:
# plotly_zone(dfm_l)
# plotly_zone(dfm_r)
bcr_l = base_calling_random(dfm_l, base_only=True)
bcr_r = base_calling_random(dfm_r, base_only=True)

mass_x (205,) mass_y (205,)
mass_x (202,) mass_y (202,)


In [146]:
plotly_basecalling(*bcr_l)
plotly_basecalling(*bcr_r)